In [78]:
import os
import sagemaker
from sagemaker.local import LocalSession
from sagemaker.predictor import RealTimePredictor, Predictor
import requests


In [3]:
sagemaker_local_session = LocalSession()
sagemaker_local_session.config = {'local': {'local_code': True}}


In [4]:
project_dir = !pwd

In [89]:
# Define SageMaker model
model = sagemaker.Model(
    model_data=f'file://model/xgb-clf-2023-05-30_20-51.tar.gz',
    # model_data='s3://crayon-task/models/xgb-clf-2023-05-30_20-51.tar.gz',
    role='arn:aws:iam::436417766157:role/service-role/AmazonSageMaker-ExecutionRole-20230529T125362',
    sagemaker_session=sagemaker_local_session,
    image_uri='test-crayon-image',
)


In [107]:
!docker build -t test-crayon-image .

Sending build context to Docker daemon  905.2kB
Step 1/8 : FROM python:3.10.4
 ---> e4ccc57bca82
Step 2/8 : WORKDIR /opt/program
 ---> Using cache
 ---> 2dba2a701bb7
Step 3/8 : COPY app/requirements.txt .
 ---> Using cache
 ---> f28178a9ff52
Step 4/8 : RUN pip install -r requirements.txt
 ---> Using cache
 ---> b1a310f960e2
Step 5/8 : COPY app .
 ---> cedeb3ca3c66
Step 6/8 : EXPOSE 8080
 ---> Running in 1dd79bff7ab7
Removing intermediate container 1dd79bff7ab7
 ---> 47b13a865bbd
Step 7/8 : ENV SM_MODEL_DIR /opt/ml/model
 ---> Running in 9dd2c7cd10e0
Removing intermediate container 9dd2c7cd10e0
 ---> 6d69e2a3d4cf
Step 8/8 : ENTRYPOINT ["gunicorn", "-b", "0.0.0.0:8080", "app:app", "-n"]
 ---> Running in ca5e2b1ae68b
Removing intermediate container ca5e2b1ae68b
 ---> c691b6641265
Successfully built c691b6641265
Successfully tagged test-crayon-image:latest


In [125]:
predictor = model.deploy(
    # predictor_cls=RealTimePredictor('test-endpoint'),
    # predictor_cls=Predictor('test-endpoint'),
    instance_type='local',
    initial_instance_count=1,
    wait=True
)

The class RealTimePredictor has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Attaching to 0vdew8lsdu-algo-1-2qe4u
0vdew8lsdu-algo-1-2qe4u | [2023-05-31 11:24:27 +0000] [1] [INFO] Starting gunicorn 20.1.0
0vdew8lsdu-algo-1-2qe4u | [2023-05-31 11:24:27 +0000] [1] [INFO] Listening at: http://0.0.0.0:8080 (1)
0vdew8lsdu-algo-1-2qe4u | [2023-05-31 11:24:27 +0000] [1] [INFO] Using worker: sync
0vdew8lsdu-algo-1-2qe4u | [2023-05-31 11:24:27 +0000] [8] [INFO] Booting worker with pid: 8
!

In [120]:
url = 'http://localhost:8080/invocations'

data = '''Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
41,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,2,Female,94,3,2,Sales Executive,4,Single,5993,19479,8,Y,Yes,11,3,1,80,0,8,0,1,6,4,0,5
49,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,3,Male,61,2,2,Research Scientist,2,Married,5130,24907,1,Y,No,23,4,4,80,1,10,3,3,10,7,1,7
37,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,4,Male,92,2,1,Laboratory Technician,3,Single,2090,2396,6,Y,Yes,15,3,2,80,0,7,3,3,0,0,0,0
'''

response = requests.post(url, data=data, headers={'Content-Type': 'text/csv'})

In [121]:
response.text

'[1, 0, 1]'

In [128]:
requests.get('http://localhost:8080/ping').text

'pong'